In [ ]:
import pandas as pd
import numpy as np

from elasticsearch import helpers
from collections.abc import MutableMapping

In [ ]:
def flatten(nested_dict, parent_key=""):
    items = []
    for k, v in nested_dict.items():
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, k).items())
        else:
            items.append((k, v))
    return dict(items)


query = {
    "sort": [{"timestamp": "desc"}],
    "query": {"match_phrase": {"event": "Search"}},
    "size": 100000,
}

response = es.search(body=query, index="search_relevance_implicit")

df = pd.DataFrame([flatten(event["_source"]) for event in response["hits"]["hits"]])

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
# grab only the columns needed
df2 = df[["query", "timestamp", "anonymousId", "network", "event"]]

# note: Python client automatically indexes from latest to earliest.  To check start date:

sorted = df2.sort_values(by=["timestamp"], ascending=True)
sorted.head(2)

In [ ]:
# data cleansing
df3 = df2.loc[(df2["network"] != "StaffCorporateDevices") & (df2["event"] == "Search")]

# note: Python client automatically indexes from latest to earliest.  To check start date:  'Search result selected']
df3.head()

In [ ]:
# count searches and only keep data for users who search 3 times or more
counts = df3.groupby("anonymousId").count()[["query"]]
counts2 = counts.loc[(counts["query"] > 3)]
# counts2.head()

In [ ]:
counts3 = counts2.drop(columns=["query"])
# counts3.head()

In [ ]:
# merge
searches_3plus = counts3.merge(df3, how="left", on="anonymousId")


pd.to_datetime(stamps, format="%Y%m%d:%H:%M:%S.%f").sort_values()

searches_3plus.sort_values(by="timestamp", ascending=False)
searches_3plus.head()

In [ ]:
# output data to csv to check

searches_3plus.to_csv("searches_3plus.csv")